# %% [markdown]
# # Project Genesis - Phase 1: Exploration Notebook
# 
# Este notebook é nosso playground para testar ideias iniciais de geração de código com IA.
# %%
# Instalações iniciais (normalmente já fizemos via requirements.txt)
# !pip install transformers torch datasets numpy

# %%
# Importações essenciais
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import numpy as np

# Verifica se temos uma GPU (acelera MUITO o treinamento)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo de treinamento: {device}")

# %% [markdown]
# ## 1. Carregando um Dataset de Exemplo
# 
# Vamos usar o `CodeSearchNet` (ou um subconjunto) para treinar nosso modelo. Ele contém pares de (função, docstring).

# %%
# Carrega um dataset pequeno para experimentação
# Em um projeto real, usaríamos o dataset completo ou um customizado
try:
    dataset = load_dataset("code_search_net", "python", split='train[:1%]') # Pega 1% dos dados para teste
    print(f"Dataset carregado. Amostras: {len(dataset)}")
    print(f"Exemplo de entrada: {dataset[0]['func_code_string'][:200]}...")  # Código
    print(f"Exemplo de docstring: {dataset[0]['func_documentation_string'][:100]}...") # Descrição
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    # Fallback para um dataset mais simples se houver erro
    dataset = load_dataset("lvwerra/codeparrot-clean", split='train[:1000]')
    print(f"Usando dataset fallback. Amostras: {len(dataset)}")

# %% [markdown]
# ## 2. Pré-processamento dos Dados
# 
# Precisamos tokenizar o texto (transformar palavras em números que o modelo entende).

# %%
# Escolha um modelo de linguagem pré-treinado como ponto de partida.
# 'microsoft/DialoGPT-small' é pequeno e bom para testes. Em produção, usaríamos um modelo de código.
model_checkpoint = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token # Define o token de padding

# Função para tokenizar os exemplos do dataset
def tokenize_function(examples):
    # Combine a docstring (objetivo) com o código (resposta) para criar o texto de entrada.
    # Formato: "Gere um código que: <DOCSTRING> \n Código: <CODE>"
    texts = [f"Gere um código que: {doc} \n Código: {code}" for doc, code in zip(examples['func_documentation_string'], examples['func_code_string'])]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

# Aplica a tokenização a todo o dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Divide o dataset em treino e teste
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# %% [markdown]
# ## 3. Carregando e Configurando o Modelo de IA

# %%
# Carrega um modelo de linguagem causal (para geração de texto)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

# %% [markdown]
# ## 4. Configurando o Treinamento

# %%
# Define os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./outputs/models/test_trainer",  # Onde salvar os checkpoints
    evaluation_strategy="epoch",                 # Avalia a cada epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,              # Tamanho do lote (reduza se faltar memória)
    per_device_eval_batch_size=4,
    num_train_epochs=1,                         # Apenas 1 epoch para teste RÁPIDO
    weight_decay=0.01,
    logging_dir="./outputs/logs/",              # Para logs do TensorBoard
    logging_steps=10,
)

# Cria o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# %% [markdown]
# ## 5. Treinamento! (Isso vai demorar)
# 
# **AVISO:** Este é um treinamento de TESTE. O modelo resultante será muito ruim, mas provará o conceito.

# %%
# Inicia o treinamento
print("Iniciando treinamento...")
trainer.train()

# Salva o modelo e o tokenizer após o treinamento
trainer.save_model("./outputs/models/first_test_model")
tokenizer.save_pretrained("./outputs/models/first_test_model")
print("Modelo salvo em './outputs/models/first_test_model'")

# %% [markdown]
# ## 6. Teste de Geração de Código
# 
# Vamos ver o que nosso modelo "aprendeu".

# %%
# Função para gerar código a partir de um prompt
def generate_code(prompt, model, tokenizer, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Prompt de teste
test_prompt = "Gere um código que: Calculate the sum of a list of numbers. \n Código:"
generated_code = generate_code(test_prompt, model, tokenizer)
print("Código gerado:\n", generated_code)

In [ ]:
# %% [markdown]
# # Project Genesis - Phase 1: Exploration Notebook
# 
# Este notebook é nosso playground para testar ideias iniciais de geração de código com IA.
# %%
# Instalações iniciais (normalmente já fizemos via requirements.txt)
# !pip install transformers torch datasets numpy

# %%
# Importações essenciais
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import numpy as np

# Verifica se temos uma GPU (acelera MUITO o treinamento)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo de treinamento: {device}")

# %% [markdown]
# ## 1. Carregando um Dataset de Exemplo
# 
# Vamos usar o `CodeSearchNet` (ou um subconjunto) para treinar nosso modelo. Ele contém pares de (função, docstring).

# %%
# Carrega um dataset pequeno para experimentação
# Em um projeto real, usaríamos o dataset completo ou um customizado
try:
    dataset = load_dataset("code_search_net", "python", split='train[:1%]') # Pega 1% dos dados para teste
    print(f"Dataset carregado. Amostras: {len(dataset)}")
    print(f"Exemplo de entrada: {dataset[0]['func_code_string'][:200]}...")  # Código
    print(f"Exemplo de docstring: {dataset[0]['func_documentation_string'][:100]}...") # Descrição
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    # Fallback para um dataset mais simples se houver erro
    dataset = load_dataset("lvwerra/codeparrot-clean", split='train[:1000]')
    print(f"Usando dataset fallback. Amostras: {len(dataset)}")

# %% [markdown]
# ## 2. Pré-processamento dos Dados
# 
# Precisamos tokenizar o texto (transformar palavras em números que o modelo entende).

# %%
# Escolha um modelo de linguagem pré-treinado como ponto de partida.
# 'microsoft/DialoGPT-small' é pequeno e bom para testes. Em produção, usaríamos um modelo de código.
model_checkpoint = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token # Define o token de padding

# Função para tokenizar os exemplos do dataset
def tokenize_function(examples):
    # Combine a docstring (objetivo) com o código (resposta) para criar o texto de entrada.
    # Formato: "Gere um código que: <DOCSTRING> \n Código: <CODE>"
    texts = [f"Gere um código que: {doc} \n Código: {code}" for doc, code in zip(examples['func_documentation_string'], examples['func_code_string'])]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

# Aplica a tokenização a todo o dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Divide o dataset em treino e teste
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# %% [markdown]
# ## 3. Carregando e Configurando o Modelo de IA

# %%
# Carrega um modelo de linguagem causal (para geração de texto)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

# %% [markdown]
# ## 4. Configurando o Treinamento

# %%
# Define os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./outputs/models/test_trainer",  # Onde salvar os checkpoints
    evaluation_strategy="epoch",                 # Avalia a cada epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,              # Tamanho do lote (reduza se faltar memória)
    per_device_eval_batch_size=4,
    num_train_epochs=1,                         # Apenas 1 epoch para teste RÁPIDO
    weight_decay=0.01,
    logging_dir="./outputs/logs/",              # Para logs do TensorBoard
    logging_steps=10,
)

# Cria o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# %% [markdown]
# ## 5. Treinamento! (Isso vai demorar)
# 
# **AVISO:** Este é um treinamento de TESTE. O modelo resultante será muito ruim, mas provará o conceito.

# %%
# Inicia o treinamento
print("Iniciando treinamento...")
trainer.train()

# Salva o modelo e o tokenizer após o treinamento
trainer.save_model("./outputs/models/first_test_model")
tokenizer.save_pretrained("./outputs/models/first_test_model")
print("Modelo salvo em './outputs/models/first_test_model'")

# %% [markdown]
# ## 6. Teste de Geração de Código
# 
# Vamos ver o que nosso modelo "aprendeu".

# %%
# Função para gerar código a partir de um prompt
def generate_code(prompt, model, tokenizer, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Prompt de teste
test_prompt = "Gere um código que: Calculate the sum of a list of numbers. \n Código:"
generated_code = generate_code(test_prompt, model, tokenizer)
print("Código gerado:\n", generated_code)